In [2]:
import warnings

warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import lightgbm as lgb

Data Read

In [5]:
datas = pd.read_csv('data/clean1.csv')

In [6]:
datas

,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label
0,54865,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827871,53,32215,4,2,112,152,28,28,28.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2827872,53,324,2,2,84,362,42,42,42.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2827873,58030,82,2,1,31,6,31,0,15.5,21.92031,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2827874,53,1048635,6,2,192,256,32,32,32.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [7]:
datas['label'].unique()

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web_Attack_Brute_Force', 'Web_Attack_XSS',
       'Web_Attack_Sql_Injection', 'FTPPatator', 'SSHPatator',
       'DoS_slowloris', 'DoS_Slowhttptest', 'DoS_Hulk', 'DoS_GoldenEye',
       'Heartbleed'], dtype=object)

In [8]:
datas.label.value_counts()

label
BENIGN                      2271320
DoS_Hulk                     230124
PortScan                     158804
DDoS                         128025
DoS_GoldenEye                 10293
FTPPatator                     7935
SSHPatator                     5897
DoS_slowloris                  5796
DoS_Slowhttptest               5499
Bot                            1956
Web_Attack_Brute_Force         1507
Web_Attack_XSS                  652
Infiltration                     36
Web_Attack_Sql_Injection         21
Heartbleed                       11
Name: count, dtype: int64

In [9]:
df_BENIGN = datas[datas['label']=='BENIGN']
df_Attack = datas[datas['label']!='BENIGN']
df_BENIGN = df_BENIGN.sample(n=None, frac=0.1, replace=False, weights=None, random_state=None, axis=0).reset_index(drop=True)

In [ ]:
minor_lb_cnt = datas.label.value_counts()
minor_lb = minor_lb_cnt[minor_lb_cnt < 100000].index

In [ ]:
minor_attack = datas[datas['label'].isin(minor_lb)]
minor_attack.label.value_counts()

In [ ]:
cp = df_Attack.copy()
#cp.drop(cp[cp['label'].isin(minor_lb)], axis=1, inplace=True)
df_MajorAttack = cp[cp['label'].isin(minor_lb)==False]

In [ ]:
minor_attack['label'] = "etc"
minor_attack['label'].value_counts()

In [10]:
#df = df_BENIGN.merge(minor_attack)
df = pd.concat([df_BENIGN, df_Attack])
#df = pd.concat([df_BENIGN, minor_attack, df_MajorAttack])
#df_external_minor = pd.concat([df_BENIGN, df_MajorAttack])


In [11]:
df.reset_index(inplace=False)
df

,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label
0,443,571122,9,8,1629,4367,1045,0,181.000000,343.470159,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,53,327628,1,1,53,93,53,53,53.000000,0.000000,...,40,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,36594,42,2,0,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,443,269411,6,6,356,3782,212,0,59.333333,89.224810,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,53,291,2,2,82,114,41,41,41.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827779,80,11512204,8,5,326,11632,326,0,40.750000,115.258405,...,32,892.0,0.0,892,892,6507197.0,0.0,6507197,6507197,DoS_GoldenEye
2827786,80,11513325,5,5,471,3525,471,0,94.200000,210.637604,...,32,918.0,0.0,918,918,6508582.0,0.0,6508582,6508582,DoS_GoldenEye
2827825,80,11509201,7,6,314,11632,314,0,44.857143,118.680845,...,32,899.0,0.0,899,899,6503248.0,0.0,6503248,6503248,DoS_GoldenEye
2827854,80,11509095,8,5,369,11632,369,0,46.125000,130.461201,...,32,914.0,0.0,914,914,6504954.0,0.0,6504954,6504954,DoS_GoldenEye


In [ ]:
del datas, df_BENIGN, df_Attack, minor_attack, minor_lb, minor_lb_cnt

In [ ]:
df.to_csv('data/clean_for_nex2.csv', index=False)

In [ ]:
df_external_minor.to_csv('data/clean_for_nex1_external_minor.csv', index=False)

In [ ]:
df = pd.read_csv('data/clean_for_nex1.csv')

In [ ]:
df.shape

In [ ]:
df.columns

In [12]:
df.drop(["destination_port"], axis=1, inplace=True)

In [13]:
df.shape

(783688, 78)

In [14]:
df.isnull().values.any()

False

In [15]:
df['label'].value_counts()

label
DoS_Hulk                    230124
BENIGN                      227132
PortScan                    158804
DDoS                        128025
DoS_GoldenEye                10293
FTPPatator                    7935
SSHPatator                    5897
DoS_slowloris                 5796
DoS_Slowhttptest              5499
Bot                           1956
Web_Attack_Brute_Force        1507
Web_Attack_XSS                 652
Infiltration                    36
Web_Attack_Sql_Injection        21
Heartbleed                      11
Name: count, dtype: int64

In [16]:
label_mapping = df['label'].values
unique_val = np.array(label_mapping)
np.unique(unique_val)

array(['BENIGN', 'Bot', 'DDoS', 'DoS_GoldenEye', 'DoS_Hulk',
       'DoS_Slowhttptest', 'DoS_slowloris', 'FTPPatator', 'Heartbleed',
       'Infiltration', 'PortScan', 'SSHPatator', 'Web_Attack_Brute_Force',
       'Web_Attack_Sql_Injection', 'Web_Attack_XSS'], dtype=object)

In [17]:
df_cp = df.copy()
labelencoder = LabelEncoder()

df_cp.iloc[:,-1] = labelencoder.fit_transform(df.iloc[:,-1])

df_cp

,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,bwd_packet_length_max,...,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label
0,571122,9,8,1629,4367,1045,0,181.000000,343.470159,1460,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,327628,1,1,53,93,53,53,53.000000,0.000000,93,...,40,0.0,0.0,0,0,0.0,0.0,0,0,0
2,42,2,0,0,0,0,0,0.000000,0.000000,0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
3,269411,6,6,356,3782,212,0,59.333333,89.224810,1460,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
4,291,2,2,82,114,41,41,41.000000,0.000000,57,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827779,11512204,8,5,326,11632,326,0,40.750000,115.258405,10184,...,32,892.0,0.0,892,892,6507197.0,0.0,6507197,6507197,3
2827786,11513325,5,5,471,3525,471,0,94.200000,210.637604,2077,...,32,918.0,0.0,918,918,6508582.0,0.0,6508582,6508582,3
2827825,11509201,7,6,314,11632,314,0,44.857143,118.680845,5792,...,32,899.0,0.0,899,899,6503248.0,0.0,6503248,6503248,3
2827854,11509095,8,5,369,11632,369,0,46.125000,130.461201,10184,...,32,914.0,0.0,914,914,6504954.0,0.0,6504954,6504954,3


In [18]:

#https://github.com/Western-OC2-Lab/AutoML-Implementation-for-Static-and-Dynamic-Data-Analytics/blob/main/AutoML_Online_Learning_Dataset_2.ipynb
# Remove irrelevant features and select important features
def Feature_Importance_IG(data):

    features = data.drop(['label'],axis=1).values
    labels = data['label'].values

    # Extract feature names
    feature_names = list(data.drop(['label'],axis=1).columns)

    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    model = lgb.LGBMRegressor(verbose = -1)
    model.fit(features, labels)
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': model.feature_importances_})

    # Sort features according to importance
    feature_importances = feature_importances.sort_values('importance', ascending = False).reset_index(drop = True)

    # Normalize the feature importances to add up to one
    feature_importances['normalized_importance'] = feature_importances['importance'] / feature_importances['importance'].sum()
    feature_importances['cumulative_importance'] = np.cumsum(feature_importances['normalized_importance'])

    cumulative_importance=0.90 # Only keep the important features with cumulative importance scores>=90%. It can be changed.

    # Make sure most important features are on top
    feature_importances = feature_importances.sort_values('cumulative_importance')

    # Identify the features not needed to reach the cumulative_importance
    record_low_importance = feature_importances[feature_importances['cumulative_importance'] > cumulative_importance]

    to_drop = list(record_low_importance['feature'])
    print(feature_importances.drop(['importance'],axis=1))
    return to_drop

In [19]:
# https://github.com/Western-OC2-Lab/AutoML-Implementation-for-Static-and-Dynamic-Data-Analytics/blob/main/AutoML_Online_Learning_Dataset_2.ipynb
# Remove redundant features
def Feature_Redundancy_Pearson(data):
    correlation_threshold = 0.90  # Only remove features with the redundancy > 90%
    features = data.drop(['label'], axis=1)
    corr_matrix = features.corr()

    # Extract the upper triangle of the correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))

    # Select the features with correlations above the threshold
    # Need to use the absolute value
    to_drop = [column for column in upper.columns if any(upper[column].abs() > correlation_threshold)]

    # Dataframe to hold correlated pairs
    record_collinear = pd.DataFrame(columns=['drop_feature', 'corr_feature', 'corr_value'])

    # Iterate through the columns to drop
    for column in to_drop:
        # Find the correlated features
        corr_features = list(upper.index[upper[column].abs() > correlation_threshold])

        # Find the correlated values
        corr_values = list(upper[column][upper[column].abs() > correlation_threshold])
        drop_features = [column for _ in range(len(corr_features))]

        # Record the information (need a temp df for now)
        temp_df = pd.DataFrame.from_dict({'drop_feature': drop_features,
                                          'corr_feature': corr_features,
                                          'corr_value': corr_values})
        record_collinear = pd.concat([record_collinear, temp_df], ignore_index=True)

    print(record_collinear)
    return to_drop


In [20]:
def Auto_Feature_Engineering(df):
    drop1 = Feature_Importance_IG(df)
    dfh1 = df.drop(columns = drop1)

    drop2 = Feature_Redundancy_Pearson(dfh1)
    dfh2 = dfh1.drop(columns = drop2)

    return dfh2

In [21]:
df2 = Auto_Feature_Engineering(df_cp)
df2

                    feature  normalized_importance  cumulative_importance
0    init_win_bytes_forward               0.139667               0.139667
1   init_win_bytes_backward               0.102667               0.242333
2             bwd_packets/s               0.053667               0.296000
3               fwd_iat_min               0.046000               0.342000
4              flow_iat_min               0.041333               0.383333
..                      ...                    ...                    ...
59        subflow_bwd_bytes               0.000000               1.000000
58        subflow_fwd_bytes               0.000000               1.000000
75            bwd_psh_flags               0.000000               1.000000
65       bwd_avg_bytes/bulk               0.000000               1.000000
76        fwd_avg_bulk_rate               0.000000               1.000000

[77 rows x 3 columns]
                   drop_feature            corr_feature  corr_value
0        total_backwa

,flow_duration,total_fwd_packets,total_length_of_fwd_packets,fwd_packet_length_max,fwd_packet_length_mean,bwd_packet_length_max,bwd_packet_length_min,flow_bytes/s,flow_iat_mean,flow_iat_min,...,bwd_packets/s,min_packet_length,psh_flag_count,urg_flag_count,down/up_ratio,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,label
0,571122,9,1629,1045,181.000000,1460,2,10498.632520,3.569512e+04,2,...,14.007515,0,1,0,0,8192,65535,8,20,0
1,327628,1,53,53,53.000000,93,93,445.627358,3.276280e+05,327628,...,3.052242,53,0,0,1,-1,-1,0,40,0
2,42,2,0,0,0.000000,0,0,0.000000,4.200000e+01,42,...,0.000000,0,0,0,0,340,-1,0,32,0
3,269411,6,356,212,59.333333,1460,0,15359.432243,2.449191e+04,13,...,22.270806,0,1,0,1,8192,60,5,20,0
4,291,2,82,41,41.000000,57,57,673539.518900,9.700000e+01,2,...,6872.852234,41,0,0,1,-1,-1,1,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827779,11512204,8,326,326,40.750000,10184,0,1038.723775,9.593503e+05,1,...,0.434322,0,1,0,0,29200,235,1,32,3
2827786,11513325,5,471,471,94.200000,2077,0,347.076105,1.279258e+06,53,...,0.434279,0,1,0,1,29200,235,1,32,3
2827825,11509201,7,314,314,44.857143,5792,0,1037.952157,9.591001e+05,46,...,0.521322,0,1,0,0,29200,235,1,32,3
2827854,11509095,8,369,369,46.125000,10184,0,1042.740546,9.590912e+05,47,...,0.434439,0,1,0,0,29200,235,1,32,3


In [22]:
indices_to_keep = ~df2.isin([np.nan, np.inf, -np.inf]).any(axis=1)
df2 = df2[indices_to_keep]
#dfh2.to_csv(directory+'dataset_FS.csv', index=0)

df2

,flow_duration,total_fwd_packets,total_length_of_fwd_packets,fwd_packet_length_max,fwd_packet_length_mean,bwd_packet_length_max,bwd_packet_length_min,flow_bytes/s,flow_iat_mean,flow_iat_min,...,bwd_packets/s,min_packet_length,psh_flag_count,urg_flag_count,down/up_ratio,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,label
0,571122,9,1629,1045,181.000000,1460,2,10498.632520,3.569512e+04,2,...,14.007515,0,1,0,0,8192,65535,8,20,0
1,327628,1,53,53,53.000000,93,93,445.627358,3.276280e+05,327628,...,3.052242,53,0,0,1,-1,-1,0,40,0
2,42,2,0,0,0.000000,0,0,0.000000,4.200000e+01,42,...,0.000000,0,0,0,0,340,-1,0,32,0
3,269411,6,356,212,59.333333,1460,0,15359.432243,2.449191e+04,13,...,22.270806,0,1,0,1,8192,60,5,20,0
4,291,2,82,41,41.000000,57,57,673539.518900,9.700000e+01,2,...,6872.852234,41,0,0,1,-1,-1,1,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827779,11512204,8,326,326,40.750000,10184,0,1038.723775,9.593503e+05,1,...,0.434322,0,1,0,0,29200,235,1,32,3
2827786,11513325,5,471,471,94.200000,2077,0,347.076105,1.279258e+06,53,...,0.434279,0,1,0,1,29200,235,1,32,3
2827825,11509201,7,314,314,44.857143,5792,0,1037.952157,9.591001e+05,46,...,0.521322,0,1,0,0,29200,235,1,32,3
2827854,11509095,8,369,369,46.125000,10184,0,1042.740546,9.590912e+05,47,...,0.434439,0,1,0,0,29200,235,1,32,3


In [23]:
df2['label'].value_counts()

label
4     230124
0     227132
10    158804
2     128025
3      10293
7       7935
11      5897
6       5796
5       5499
1       1956
12      1507
14       652
9         36
13        21
8         11
Name: count, dtype: int64

In [24]:
df2['label'] = pd.to_numeric(df2['label'], errors='coerce').astype('int')
df2['label'].dtype

dtype('int32')

In [25]:
df2

,flow_duration,total_fwd_packets,total_length_of_fwd_packets,fwd_packet_length_max,fwd_packet_length_mean,bwd_packet_length_max,bwd_packet_length_min,flow_bytes/s,flow_iat_mean,flow_iat_min,...,bwd_packets/s,min_packet_length,psh_flag_count,urg_flag_count,down/up_ratio,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,label
0,571122,9,1629,1045,181.000000,1460,2,10498.632520,3.569512e+04,2,...,14.007515,0,1,0,0,8192,65535,8,20,0
1,327628,1,53,53,53.000000,93,93,445.627358,3.276280e+05,327628,...,3.052242,53,0,0,1,-1,-1,0,40,0
2,42,2,0,0,0.000000,0,0,0.000000,4.200000e+01,42,...,0.000000,0,0,0,0,340,-1,0,32,0
3,269411,6,356,212,59.333333,1460,0,15359.432243,2.449191e+04,13,...,22.270806,0,1,0,1,8192,60,5,20,0
4,291,2,82,41,41.000000,57,57,673539.518900,9.700000e+01,2,...,6872.852234,41,0,0,1,-1,-1,1,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827779,11512204,8,326,326,40.750000,10184,0,1038.723775,9.593503e+05,1,...,0.434322,0,1,0,0,29200,235,1,32,3
2827786,11513325,5,471,471,94.200000,2077,0,347.076105,1.279258e+06,53,...,0.434279,0,1,0,1,29200,235,1,32,3
2827825,11509201,7,314,314,44.857143,5792,0,1037.952157,9.591001e+05,46,...,0.521322,0,1,0,0,29200,235,1,32,3
2827854,11509095,8,369,369,46.125000,10184,0,1042.740546,9.590912e+05,47,...,0.434439,0,1,0,0,29200,235,1,32,3


In [63]:
label_df = {}
label_df = {
    'code': df2['label'].unique(),
    'label': labelencoder.inverse_transform(df2['label'].unique()),
    'count': df2['label'].value_counts(sort=False)
}



{'code': array([ 0,  2, 10,  1,  9, 12, 14, 13,  7, 11,  6,  5,  4,  3,  8]),
 'label': array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
        'Web_Attack_Brute_Force', 'Web_Attack_XSS',
        'Web_Attack_Sql_Injection', 'FTPPatator', 'SSHPatator',
        'DoS_slowloris', 'DoS_Slowhttptest', 'DoS_Hulk', 'DoS_GoldenEye',
        'Heartbleed'], dtype=object),
 'count': label
 0     227132
 2     128025
 10    158804
 1       1956
 9         36
 12      1507
 14       652
 13        21
 7       7935
 11      5897
 6       5796
 5       5499
 4     230124
 3      10293
 8         11
 Name: count, dtype: int64}

In [65]:
label_df = pd.DataFrame(label_df)
label_df = label_df.sort_values('count', ascending=False).reset_index(drop = True)
label_df

,code,label,count
0,4,DoS_Hulk,230124
1,0,BENIGN,227132
2,10,PortScan,158804
3,2,DDoS,128025
4,3,DoS_GoldenEye,10293
5,7,FTPPatator,7935
6,11,SSHPatator,5897
7,6,DoS_slowloris,5796
8,5,DoS_Slowhttptest,5499
9,1,Bot,1956


In [67]:
label_df.to_csv('data/classes/label_code_count.csv')